# Podatkovni model za aplikacijo

Za potrebe spletne aplikacije smo pripravili ogrodje podatkovnega modela, ki skrbi za komunikacijo z relacijsko podatkovno bazo in predstavitev entitet v bazi z objekti. Takemu pristopu pravimo *objektno-relacijsko preslikovanje* (*ORM*, angl. *object-relational mapping*).

In [1]:
from model import Kraj, Oseba, Racun, Transakcija, vzpostavi_povezavo, ustvari_bazo, uvozi_podatke
conn = vzpostavi_povezavo()

Pripravimo si prvi objekt in ga shranimo v bazo.

In [2]:
lj = Kraj.ustvari(1000, 'Ljubljana')
lj.shrani()

Izpišimo seznam krajev iz baze.

In [3]:
list(Kraj.seznam())

[<Kraj: 1000 Ljubljana>]

Pripravimo še ostale objekte. Pri shranjevanju v bazo se bodo shranili tudi povezani objekti.

In [4]:
mb = Kraj.ustvari(2000, 'Maribor')

janez = Oseba.ustvari('Janez', 'Novak', '1234567890123', 'Tržaška cesta', lj)
micka = Oseba.ustvari('Micka', 'Kovač', '9876543210987', 'Ljubljanska cesta', mb)
peter = Oseba.ustvari('Peter', 'Klepec', '1029384756601', 'Klepčeva ulica', 1000)

racun_j1 = Racun.ustvari(oseba_id=janez)
racun_m = Racun.ustvari(oseba_id=micka)
racun_j2 = Racun.ustvari(oseba_id=janez)
racun_p = Racun.ustvari(oseba_id=peter)

with conn.transaction():
    Transakcija.ustvari(racun_id=racun_j1, znesek=100, opis='odprtje računa').shrani()
    Transakcija.ustvari(racun_id=racun_m, znesek=500, opis='prihranki').shrani()
    Transakcija.ustvari(racun_id=racun_j2, znesek=10, opis='varčevanje').shrani()
    Transakcija.ustvari(racun_id=racun_j1, znesek=-15, opis='dvig').shrani()
    Transakcija.ustvari(racun_id=racun_p, znesek=1000).shrani()

In [5]:
list(Oseba.seznam())

[<Oseba: Micka Kovač (9876543210987)>,
 <Oseba: Janez Novak (1234567890123)>,
 <Oseba: Peter Klepec (1029384756601)>]

In [6]:
list(Transakcija.seznam())

[<Transakcija: Transakcija 1 na računu 1 z zneskom 100>,
 <Transakcija: Transakcija 2 na računu 2 z zneskom 500>,
 <Transakcija: Transakcija 3 na računu 3 z zneskom 10>,
 <Transakcija: Transakcija 4 na računu 1 z zneskom -15>,
 <Transakcija: Transakcija 5 na računu 4 z zneskom 1000>]

Do vrednostih posameznih atributov v objektih dostopamo z istoimenskimi polji. Če gre za referenco, bomo tako dobili ustrezen objekt.

In [7]:
peter.naslov

'Klepčeva ulica'

In [8]:
peter.kraj_id

<Kraj: 1000 Ljubljana>

Z indeksiranjem dobimo vrednosti iz baze - tj., v primeru referenc dobimo ustrezno vrednost ključa.

In [9]:
tr = Transakcija.z_id(5)
tr['cas']

datetime.datetime(2024, 3, 25, 16, 40, 11, 304365)

In [10]:
tr['racun_id']

4

Vrednosti lahko tudi spreminjamo ter nato spremembe shranimo v bazo.

In [11]:
tr.opis = 'dobitek na lotu'
tr.shrani()

In [12]:
for tid, znesek, racun, cas, opis in Transakcija.seznam():
    print(f"{tid}: znesek {znesek} na računu {racun} osebe {Racun.z_id(racun).oseba_id}: {opis} ({cas})")

1: znesek 100 na računu 1 osebe Janez Novak (1234567890123): odprtje računa (2024-03-25 16:40:11.304365)
2: znesek 500 na računu 2 osebe Micka Kovač (9876543210987): prihranki (2024-03-25 16:40:11.304365)
3: znesek 10 na računu 3 osebe Janez Novak (1234567890123): varčevanje (2024-03-25 16:40:11.304365)
4: znesek -15 na računu 1 osebe Janez Novak (1234567890123): dvig (2024-03-25 16:40:11.304365)
5: znesek 1000 na računu 4 osebe Peter Klepec (1029384756601): dobitek na lotu (2024-03-25 16:40:11.304365)


**Pozor**: tak način ni učinkovit, saj se pri vsaki transakciji izvedeta še poizvedbi za račun in za osebo. Če bomo v aplikaciji želeli tak izpis, bomo implementirali ustrezno metodo, ki bo iz baze pridobila želene podatke s čim manjšim številom poizvedb.

## Uvoz podatkov iz datotek

Pobrišimo tabele in jih napolnimo s podatki, ki smo jih generirali s spletnim orodjem [Mockaroo](https://www.mockaroo.com).

In [13]:
ustvari_bazo(izbrisi=True)
uvozi_podatke()

Oglejmo si nekaj uvoženih podatkov.

In [14]:
for _, oseba in zip(range(10), Oseba.seznam()):
    print(oseba)

Karoly Yoseloff (1)
Silvia Cammell (2)
Gabie Grigoryov (3)
Bari Kemmis (4)
Harlan Jendas (5)
Angelica Kinnerley (6)
Sheila-kathryn MacCostye (7)
Lynnea Perle (8)
Dalston Grieves (9)
Phineas Nolan (10)
